### Preliminars

##### Imports

In [ ]:
import preprocess
import sample
import cluster
import nn
import k_means
import fuzzy_c_means
import mountain
import regression

import numpy as np
import matplotlib.pyplot as plt
import itertools as iter
import os

%matplotlib widget
%reload_ext autoreload
%autoreload 2

np.random.seed(420)
x_size, y_size = 4, 4
def fig_size(y, x): return (x*x_size, y*y_size)
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

os.chdir("../images/")


##### Read Data

In [ ]:
dataset = 0
if dataset == 0:
    file = "../data/pulsar_data_train.csv"
    index_col, header = None, 0
    classify = True
elif dataset == 1:
    file = "../data/IRIS.csv"
    index_col, header = 0, 0
    classify = True
else:
    file = '../data/examen.csv'
    index_col, header = None, None
    classify = False

data, N_S, axes_names, indices = preprocess.get_data(file, header, index_col)

X_S, Y_S, revert_X, revert_Y = preprocess.preprocess(data, classify)


# Sample

### PDF properties

In [ ]:
pdf_info = sample.pdf_info(indices)


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=fig_size(1, 2))
fig.suptitle("Sample PDF analytical comparison")
ax[0].set_title("Probability Density Function")
ax[1].set_title("Information content")
cluster.axis_labels(ax[0], "Sample Index", "$f(x)$")
cluster.axis_labels(ax[1], "Sample Index", "$I(x)$")

for name, info in pdf_info.items():
    ax[0].plot(indices, info["F"], label=f"{name}: {info['H']:.2f}")
    ax[1].plot(indices, info["I"])
fig.legend(title="Entropy $H(X)$", loc="lower center")
fig.savefig("sample:info.png")


### Sampling data

In [ ]:
pdf_sample = sample.sample(indices)
partition_index = pdf_sample["Uniform"]
partition = {k: (X_S[:, v], Y_S[:, v]) for k, v in partition_index.items()}


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=fig_size(1, 3))
fig.suptitle("Sample PDF numerical comparison")
for ax_pdf, (pdf_name, pdf_p) in zip(ax, pdf_sample.items()):
    ax_pdf.set_title(f"{pdf_name} Sampling")
    cluster.axis_labels(ax_pdf, 'Index', 'Sample Count')
    ax_pdf.hist(pdf_p.values(), stacked=True, bins=10)
    if not cluster:
        continue
    legend = []

    for c in range(len(revert_Y)):
        legend.append(f"{revert_Y[c]}: " + 
            " / ".join([f"{np.sum(Y_S[c, p_i])/np.sum(Y_S[c,:] == 1):.0%}"
            for p_name, p_i in pdf_p.items()]))

    legend = ax_pdf.legend(legend, title="Cluster distribution", loc="lower left", handlelength=0, handletextpad=0)
legend = fig.legend([f"{k}: {len(v)} ({len(v)/N_S:.0%})"
                     for k, v in partition_index.items()] + [f"Total: {N_S}"],
                    title="Data Distribution", loc="upper right")
legend.legendHandles[-1].set_visible(False)
fig.savefig("sample:hist.png")


### Data Distribution

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})
cluster.plot(ax, X_S, Y_S, title="Data Distribution", cluster_names=revert_Y)
fig.savefig("sample:dist:train.png")

# Supervised Learning

### Linear Regression

In [ ]:
B_linear, E_linear = regression.linear(*partition["Train"])
B_logistic, E_logistic = regression.logistic(*partition["Train"])


In [ ]:
for name, (X, Y) in partition.items():
    fig, ax = plt.subplots(1, 2, figsize=fig_size(
        1, 2), subplot_kw={'projection': '3d'})
    fig.suptitle(f"Regression Output ({name})")
    U_linear = regression.linear_eval(X, B_linear)
    cluster.plot(ax[0], X, U_linear, Y,
                 title=f"Linear",
                 cluster_names=revert_Y, axes_names=axes_names)
    U_logistic = regression.logistic_eval(X, B_logistic)
    cluster.plot(ax[1], X, U_logistic, Y,
                 title=f"Logistic",
                 cluster_names=revert_Y, axes_names=axes_names)
    fig.savefig(f"linear:{name.lower()}.png")


### Neural Network

In [ ]:
epochs = 1000
neurons = np.array(range(5))+1
layers = np.array(range(3))+1
eta = [0.2, 0.5, 0.9]

nn_metric = {
    "grad:min": (np.inf, None, None), 
    "grad:max": (-np.inf, None, None),
    "err:min" : (np.inf, None, None),
    "err:max" : (-np.inf, None, None)
}

for params in iter.product(neurons, layers, eta):   
    c_neurons, c_layers, c_eta = params
    print(params)
    with HiddenPrints():
        W, B, delta_nn, E_nn = nn.train(partition["Train"], epochs=epochs, 
            hidden=[c_neurons] * c_layers, eta=c_eta, classify=classify, test_set=partition)
    grad = np.sum(delta_nn[-1])
    err = np.sum([e[-1] for e in E_nn.values()])

    if grad < nn_metric["grad:min"][0]:
        nn_metric["grad:min"] = (grad, params, (W, B, delta_nn, E_nn))
    elif grad > nn_metric["grad:max"][0]:
        nn_metric["grad:max"] = (grad, params, (W, B, delta_nn, E_nn))
    if err < nn_metric["err:min"][0]:
        nn_metric["err:min"] = (err, params, (W, B, delta_nn, E_nn))
    elif err > nn_metric["err:max"][0]:
        nn_metric["err:max"] = (err, params, (W, B, delta_nn, E_nn))

In [ ]:
skip = 10
nn_title = {
    "grad:min": "Min Gradient",
    "grad:max": "Max Gradient",
    "err:min": "Min Error",
    "err:max": "Max Error"
}
for name, (value, (neurons, layers, eta), ((W, B, delta_nn, E_nn))) in nn_metric.items():
    fig, ax = plt.subplots(1, 2, figsize=fig_size(1, 2))
    fig.suptitle(f"{nn_title[name]}: {neurons, layers, eta}")
    ax[0].set_title("Gradients per epoch")
    ax[0].plot(range(skip, epochs), delta_nn[skip:],
            label=[f"{l}: {d:.2e}" for l, d in enumerate(delta_nn[-1])])
    ax[0].plot([], [], label = f"Total: {np.sum(delta_nn[-1]):.2e}")
    legend = ax[0].legend(title="Layer", loc="upper right")
    legend.legendHandles[-1].set_visible(False)
    cluster.axis_labels(ax[0], "Epoch",
                        "Average Length of Gradients $\\overline{\\Vert\\delta_l\\Vert_2}$")

    ax[1].set_title("Error per epoch")
    for p_name in E_nn.keys():
        ax[1].plot(range(skip, epochs), E_nn[p_name][skip:],
                label=f"{p_name}: {E_nn[p_name][-1]:.2e}")
    ax[1].plot([], [], label = f"Total: {np.sum([e[-1] for e in E_nn.values()]):.2e}")
    legend = ax[1].legend(title="Partition", loc="upper right")
    legend.legendHandles[-1].set_visible(False)
    cluster.axis_labels(ax[1], "Epoch",
                        "Average Error $\\frac{\\overline{\\Vert Y_d - Y_k\\Vert_2}}{2}$")
    fig.savefig(f"nn:{name}.png")

In [ ]:
for name, (value, (neurons, layers, eta), (W, B, delta_nn, E_nn)) in nn_metric.items():
    print(name)
    with HiddenPrints():
        W, B, delta_nn2, E_nn2 = nn.train(partition["Train"], epochs=epochs,
           hidden=[neurons] * layers, eta=c_eta, classify=classify, test_set=partition, W=W, B=B)
    delta_nn.extend(delta_nn2)
    for k in E_nn:
        E_nn[k].extend(E_nn2[k])
    grad = np.sum(delta_nn[-1])
    err = np.sum([e[-1] for e in E_nn.values()])
    
    if name == "grad:min":
        nn_metric[name] = (grad, params, (W, B, delta_nn, E_nn))
    elif name == "grad:max":
        nn_metric[name] = (grad, params, (W, B, delta_nn, E_nn))
    elif name == "err:min":
        nn_metric[name] = (err, params, (W, B, delta_nn, E_nn))
    elif name == "err:max":
        nn_metric[name] = (err, params, (W, B, delta_nn, E_nn))
    


In [ ]:
for name, (value, (neurons, layers, eta), (W, B, delta_nn, E_nn)) in nn_metric.items():
    fig, ax = plt.subplots(1, 2, figsize=fig_size(1, 2))
    fig.suptitle(f"{nn_title[name]}: {params[0], params[1]+1, params[2]}")
    ax[0].set_title("Gradients per epoch")
    ax[0].plot(range(skip, epochs), delta_nn[skip:],
            label=[f"{l}: {d:.2e}" for l, d in enumerate(delta_nn[-1])])
    ax[0].plot([], [], label = f"Total: {np.sum(delta_nn[-1]):.2e}")
    legend = ax[0].legend(title="Layer", loc="upper right")
    legend.legendHandles[-1].set_visible(False)
    cluster.axis_labels(ax[0], "Epoch",
                        "Average Length of Gradients $\\overline{\\Vert\\delta_l\\Vert_2}$")

    ax[1].set_title("Error per epoch")
    for p_name in E_nn.keys():
        ax[1].plot(range(skip, epochs), E_nn[p_name][skip:],
                label=f"{p_name}: {E_nn[p_name][-1]:.2e}")
    ax[1].plot([], [], label = f"Total: {np.sum([e[-1] for e in E_nn.values()]):.2e}")
    legend = ax[1].legend(title="Partition", loc="upper right")
    legend.legendHandles[-1].set_visible(False)
    cluster.axis_labels(ax[1], "Epoch",
                        "Average Error $\\frac{\\overline{\\Vert Y_d - Y_k\\Vert_2}}{2}$")
    fig.savefig(f"nn:{name}2.png")

    fig, ax = plt.subplots(1, 3, figsize=fig_size(1, 3), 
        subplot_kw={'projection': '3d'})
    fig.suptitle(f"{nn_title[name]} Output")
    for i, (p_name, XY) in enumerate(partition.items()):
        U, E = nn.eval(XY[0], W, B, classify=classify, Yd=XY[1])
        cluster.plot(ax[i], XY[0], U, XY[1], title=p_name, cluster_names=revert_Y)

# Unsupervised Learning

### Mountain Clustering

In [ ]:
n = 10
C = mountain.run(partition["Train"][0], 1, 1, 0.1, n=n)


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=fig_size(
    1, 3), subplot_kw={'projection': '3d'})
for i, (name, XY) in enumerate(partition.items()):
    U = k_means.eval(XY[0], C)
    if U.shape[0] == XY[1].shape[0]:
        U = cluster.match(U, XY[1])
        Ud = XY[1]
    else:
        Ud = None
    cluster.plot(ax[i], XY[0], U, Ud=Ud, C=C, title=f"Mountain Clustering ({name})",
                 cluster_names=revert_Y, axes_names=axes_names)
fig.savefig("mountain:clusters.png")


### Substractive Clustering

In [ ]:
C = mountain.run(partition["Train"][0], 1.1, 1, 0.1, substractive=True)


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=fig_size(
    1, 3), subplot_kw={'projection': '3d'})
for i, (name, XY) in enumerate(partition.items()):
    U = k_means.eval(XY[0], C)
    if U.shape[0] == XY[1].shape[0]:
        U = cluster.match(U, XY[1])
        Ud = XY[1]
    else:
        Ud = None
    cluster.plot(ax[i], XY[0], U, Ud=Ud, C=C, title=f"Substractive Clustering ({name})",
                 cluster_names=revert_Y, axes_names=axes_names)
fig.savefig("substractive:clusters.png")

### K-Means

In [ ]:
U, C = k_means.run(2, partition["Train"][0], 0.002)


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=fig_size(
    1, 3), subplot_kw={'projection': '3d'})
for i, (name, XY) in enumerate(partition.items()):
    U = k_means.eval(XY[0], C)
    if U.shape[0] == XY[1].shape[0]:
        U = cluster.match(U, XY[1])
        Ud = XY[1]
    else:
        Ud = None
    cluster.plot(ax[i], XY[0], U, Ud=Ud, C=C, title=f"K-means ({name})",
                 cluster_names=revert_Y, axes_names=axes_names)
fig.savefig("kmeans:clusters.png")


### Fuzzy C-means

In [ ]:
m = 5
U, C = fuzzy_c_means.run(2, partition["Train"][0], m, 0.002)


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=fig_size(
    1, 3), subplot_kw={'projection': '3d'})
for i, (name, XY) in enumerate(partition.items()):
    U = fuzzy_c_means.eval(XY[0], C, m)
    if U.shape[0] == XY[1].shape[0]:
        U = cluster.match(U, XY[1])
        Ud = XY[1]
    else:
        Ud = None
    cluster.plot(ax[i], XY[0], U, Ud=Ud, C=C, title=f"Fuzzy C-means ({name})",
                 cluster_names=revert_Y, axes_names=axes_names)
fig.savefig("fuzzycmeans:clusters.png")
